In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# nastavíme zobrazovanie grafov priamo v odstavcoch zápisníka
%matplotlib inline
# inicializujeme knižnicu seaborn
sns.set()

In [57]:
# načítanie a transformácia dát
data = pd.read_csv("../data/titanic.csv")

# nahradenie skratiek pre miesto nalodenia
data["embarked"] = data["embarked"].map({"S": "Southampton", "Q": 'Queenstown', "C":"Cherbourg"})

# pridanie celkového počtu príbuzných a binárny atribút pre cestujúcich s rodinou
data["family"] = data.eval("sibsp + parch")
data["has_family"] = data.eval("family > 0")

# nahradíme chýbajúce hodnoty cestovného mediánom a diskretizujeme ho na 3 intervaly
data["fare"].fillna(data["fare"].median(), inplace=True)
data["fare_ordinal"] = pd.cut(data["fare"], bins=[0, 25, 100, 520], include_lowest=True, labels=["normal", "more expensive", "most expensive"])

# z mena si vyextrahujeme titul
def extract_title(name):
    if pd.isna(name):
        return np.nan 
    start = name.find(",") + 1
    end = name.find(".")
    return name[start:end].strip()

data["title"] = data["name"].apply(extract_title)
data.drop(columns="name", inplace=True)

# premapujeme titul na skrátený zoznam
def map_title(title):
    if title in {"Master", "Dr", "Rev", "Col", "Major", "Don", "Jonkheer", "Sir", "Dona", "Lady", "Capt", "the Countess"}:
        return "rare title"
    elif title in {"Mlle", "Ms"}:
        return "Miss"
    elif title in {"Mme"}:
        return "Mrs"
    return title;

data["title_short"] = data["title"].apply(map_title)

# nahradíme chýbajúceho hodnoty pre vek mediánom pre kombinácie pohlavia a titulu
ptable = pd.pivot_table(data, index=["sex", "title_short"], values="age", aggfunc="median")

def replace_missing_age(row):
    age = row["age"]
    sex = row["sex"]
    title = row["title_short"]
    if pd.isna(age):
        return ptable["age"][(sex, title)]
    else:
        return age

data["age"] = data.apply(replace_missing_age, axis=1)

### Úloha 8.1

Chýbajúce hodnoty pre atribút `embarked` nahraďte najfrekventovanejšou hodnotou.

In [58]:
data["embarked"].value_counts() # zobrazíme si rôzne hodnoty a ich početnosti

Southampton    914
Cherbourg      270
Queenstown     123
Name: embarked, dtype: int64

In [59]:
# najfkrekventovanejšia hodnota je Southhampton
data["embarked"].fillna("Southampton", inplace=True)
data["embarked"].isna().sum() # skontrolujeme počet chýbajúcich hodnôt po nahradení

0

### Úloha 8.2

Odvoďte nový atribút `age_ordinal` diskretizovaním hodnôt `age` na intervaly 0-13, 13-19, 19-65, 65-maximálny vek s označeniami `child`, `young`, `adult`, `old`.

In [60]:
# zistíme maximálnu hodnotu pre vek
max_age = data["age"].max()
# rozdelíme hodnoty na intervaly
data["age_ordinal"] = pd.cut(data["age"], bins=[0, 13, 19, 65, max_age], include_lowest=True, labels=["child", "young", "adult", "old"])
# zobrazíme ordinálne hodnoty a ich početnosti
data["age_ordinal"].value_counts()

adult    1065
young     126
child     108
old        10
Name: age_ordinal, dtype: int64

### Úloha 8.3
Pomocou krížovej tabuľky zistite, koľko mužov a žien má aký titul.

In [61]:
pd.crosstab(index=data["title"], columns=data["sex"])

sex,female,male
title,,
Capt,0,1
Col,0,4
Don,0,1
Dona,1,0
Dr,1,7
Jonkheer,0,1
Lady,1,0
Major,0,2
Master,0,61


### Úloha 8.4
Vytvorte kontingenčnú tabuľku v ktorej prehľadne zobrazíte počet zachránených pasažierov pre skupiny rozdelené podľa veku (`age_ordinal`), triedy a pohlavia. Ak použijete ako agregačnú funkciu strednú hodnotu `survived`, ako môžete výsledné čísla interpretovať?

In [62]:
pd.pivot_table(data, index=["sex", "age_ordinal"], columns="pclass", values="survived", aggfunc="mean")

pclass                     1         2         3
sex    age_ordinal                              
female child        0.000000  1.000000  0.483871
       young        1.000000  0.900000  0.593750
       adult        0.968504  0.865854  0.470588
       old          1.000000       NaN       NaN
male   child        0.833333  1.000000  0.333333
       young        0.200000  0.062500  0.083333
       adult        0.331288  0.091549  0.140704
       old          0.200000  0.000000  0.000000

### Úloha 8.5
Zistite, či má na prežitie vplyv výška cestovného, alebo miesto nalodenia. 

In [63]:
pd.pivot_table(data, index="fare_ordinal", values="survived", aggfunc="mean")

,survived
fare_ordinal,
normal,0.289538
more expensive,0.501241
most expensive,0.714286


In [64]:
pd.pivot_table(data, index="embarked", values="survived", aggfunc="mean")

,survived
embarked,
Cherbourg,0.555556
Queenstown,0.357724
Southampton,0.334061


Väčšia závislosť je medzi výškou cestovného (pravdepodobnosti prežitia sa výraznejšie odlišujú pre rôzne kategorické hodnoty).

In [65]:
# zobrazíme si kombinácie pre oba atribúty a pridáme aj celkové štatistiky pre každý riadok a stĺpec (nastavenie margins=True)
pd.pivot_table(data, index="embarked", columns="fare_ordinal", values="survived", aggfunc="mean", margins=True)

fare_ordinal,normal,more expensive,most expensive,All
embarked,,,,
Cherbourg,0.389831,0.666667,0.720000,0.555556
Queenstown,0.368421,0.222222,NaN,0.357724
Southampton,0.254237,0.452055,0.705882,0.334061
All,0.714286,0.501241,0.289538,0.381971


### Úloha 8.6

Označenia kajuty začínajú písmenom, ktoré označuje palubu na ktorej sa kajuta nachádzala (napr. kajuta `C22` sa nachádzala na palube `C`, atď.). Pomocou metódy `apply` odvoďte nový atribút `deck` s označením paluby.

Na ktorej palube by ste mali väčšiu šancu na záchranu?

In [72]:
# definujeme si funkciu pre extrahovanie hodnôt
def extract_deck(s):
    if pd.isna(s): # vrátime prázdnu hodnotu pre prázdne označenie kajuty
        return np.nan 
    return s.strip()[0] # odstránime z označenia kajuty prázdne znaky, a vrátime reťazec s prvým písmenom označujúcim palubu

# aplikujeme funkciu na hodnoty označenia kajuty a vyextrahujeme označenie paluby
data["deck"] = data["cabin"].apply(extract_deck)
# pre kontrolu si zobrazíme rôzne hodnoty a ich početnosti
data["deck"].value_counts()

C    94
B    65
D    46
E    41
A    22
F    21
G     5
T     1
Name: deck, dtype: int64

In [75]:
pd.pivot_table(data, index="deck", values="survived", aggfunc="mean")

,survived
deck,
A,0.500000
B,0.723077
C,0.606383
D,0.695652
E,0.731707
F,0.619048
G,0.600000
T,0.000000


Najväčšia pravdepodobnosť prežitia je na palube E.

In [77]:
# spracované dáta si môžete uložiť do samostatného súboru, ktorý potom môžete znovu načítať
data.to_csv("../data/titanic-processed.csv", index=False) # pri zapisovaní csv dát sa štandardne uloží aj stĺpec s indexom s číslami riadkov,
                                                          # ak ho chceme vynechať nastavíme parameter index na False